In [3]:
import pandas as pd 
import numpy as np
import time

In [21]:
N_STATES = 6   # 1维世界的宽度
ACTIONS = ['left', 'right']     # 探索者的可用动作
EPSILON = 0.9   # 贪婪度 greedy
ALPHA = 0.1     # 学习率
GAMMA = 0.9    # 奖励递减值
MAX_EPISODES = 20   # 最大回合数
FRESH_TIME = 0.3    # 移动间隔时间

In [12]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table 全 0 初始
        columns=actions,    # columns 对应的是行为名称
    )
    return table

# q_table:
#    left  right
# 0   0.0    0.0
# 1   0.0    0.0
# 2   0.0    0.0
# 3   0.0    0.0
# 4   0.0    0.0
# 5   0.0    0.0


In [6]:
# 在某个 state 地点, 选择行为
def choose_action(state, q_table):
    state_actions = q_table.iloc[state, :]  # 选出这个 state 的所有 action 值
    if (np.random.uniform() > EPSILON) or (state_actions.all() == 0):  # 非贪婪 or 或者这个 state 还没有探索过(全零return true==0)
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.idxmax()    # 贪婪模式
    return action_name

In [7]:
def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        if S == N_STATES - 2:   # terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # move left
        R = 0
        if S == 0:
            S_ = S  # reach the wall
        else:
            S_ = S - 1
    return S_, R

In [8]:
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

In [9]:
def rl():
    q_table = build_q_table(N_STATES, ACTIONS)  # 初始 q table
    for episode in range(MAX_EPISODES):     # 回合
        step_counter = 0
        S = 0   # 回合初始位置
        is_terminated = False   # 是否回合结束
        update_env(S, episode, step_counter)    # 环境更新
        while not is_terminated:

            A = choose_action(S, q_table)   # 选行为
            S_, R = get_env_feedback(S, A)  # 实施行为并得到环境的反馈
            q_predict = q_table.loc[S, A]    # 估算的(状态-行为)值
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   #  实际的(状态-行为)值 (回合没结束)
            else:
                q_target = R     #  实际的(状态-行为)值 (回合结束)
                is_terminated = True    # terminate this episode

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  #  q_table 更新
            S = S_  # 探索者移动到下一个 state

            update_env(S, episode, step_counter+1)  # 环境更新

            step_counter += 1
    return q_table


In [22]:
q_table = rl()
print('\r\nQ-table:\n')
print(q_table)

--o--T                          

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  import sys


                                
Q-table:

       left     right
0  0.000005  0.035320
1  0.001276  0.108782
2  0.000030  0.280509
3  0.009508  0.572472
4  0.051006  0.878423
5  0.000000  0.000000


In [1]:
import gym
import holdem

In [3]:
gym.make()

In [1]:
import gym
import holdem

def play_out_hand(env, n_seats):
  # reset environment, gather relevant observations
  (player_states, (community_infos, community_cards)) = env.reset()
  (player_infos, player_hands) = zip(*player_states)

  # display the table, cards and all
  env.render(mode='human')

  terminal = False
  while not terminal:
    # play safe actions, check when noone else has raised, call when raised.
    actions = holdem.safe_actions(community_infos, n_seats=n_seats)
    (player_states, (community_infos, community_cards)), rews, terminal, info = env.step(actions)
    env.render(mode='human')

env = gym.make('TexasHoldem-v1') # holdem.TexasHoldemEnv(2)

# start with 2 players
env.add_player(0, stack=2000) # add a player to seat 0 with 2000 "chips"
env.add_player(1, stack=2000) # add another player to seat 1 with 2000 "chips"
# play out a hand
play_out_hand(env, env.n_seats)

# # add one more player
# env.add_player(2, stack=2000) # add another player to seat 1 with 2000 "chips"
# # play out another hand
# play_out_hand(env, env.n_seats)

total pot: 35
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♦],[T♦] stack: 1990
1 [J♦],[2♣] stack: 1975
2 [  ],[  ] stack: 0
3 [  ],[  ] stack: 0
total pot: 50
last action by player 0:
- call, current bet: 25
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♦],[T♦] stack: 1975
1 [J♦],[2♣] stack: 1975
2 [  ],[  ] stack: 0
3 [  ],[  ] stack: 0
total pot: 50
last action by player 1:
_ check
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♦],[T♦] stack: 1975
1 [J♦],[2♣] stack: 1975
2 [  ],[  ] stack: 0
3 [  ],[  ] stack: 0
total pot: 50
last action by player 0:
_ check
community:
- [Q♦],[7♥],[5♦],[  ],[  ] 
players:
0 [9♦],[T♦] stack: 1975
1 [J♦],[2♣] stack: 1975
2 [  ],[  ] stack: 0
3 [  ],[  ] stack: 0
total pot: 50
last action by player 0:
_ check
community:
- [Q♦],[7♥],[5♦],[  ],[  ] 
players:
0 [9♦],[T♦] stack: 1975
1 [J♦],[2♣] stack: 1975
2 [  ],[  ] stack: 0
3 [  ],[  ] stack: 0
total pot: 50
last action by player 1:
_ check
community:
- [Q♦],[7♥],[5♦],[  ],[  ] 
player